In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor

from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
import pandas as pd
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from sklearn.metrics import accuracy_score

In [ ]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [ ]:
dataset = pd.read_excel('alt.xlsx', header=0, index_col=0)
values = dataset.values
# ensure all data is float
values = values.astype('float32')

# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
n_hours =24
n_features = 1
n_out=6
# frame as supervised learning
reframed = series_to_supervised(scaled, n_hours,n_out)
#reframed = series_to_supervised(scaled, n_hours)
print(reframed.head())
print(reframed.shape)
print(dataset.shape)

In [ ]:
# split into train and test sets
values = reframed.values
n_train_hours = int(43848*0.6)
n_val_hours =int(43848*0.8)
train = values[:n_train_hours, :]
val = values[n_train_hours:n_val_hours, :]
test = values[n_val_hours:, :]
n_obs = n_hours * n_features

# split into input and outputs
train_X, train_y = train[:, :n_obs], train[:,  n_obs:]
val_X, val_y = val[:, :n_obs], val[:,  n_obs:]
test_X, test_y = test[:, :n_obs], test[:,  n_obs:]

print(train_X.shape, len(train_X), train_y.shape)
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0],n_hours))
val_X = val_X.reshape((val_X.shape[0], n_hours))
test_X = test_X.reshape((test_X.shape[0], n_hours))
print(train_X.shape, train_y.shape, val_X.shape, val_y.shape,test_X.shape, test_y.shape)

In [ ]:
from sklearn.multioutput import RegressorChain
model = RegressorChain(RandomForestRegressor(n_estimators= 150, random_state=10))
model.fit(train_X, train_y)

In [ ]:
yhat = model.predict(test_X)

test_y = test_y.reshape((len(test_y),n_out))
inv_y = scaler.inverse_transform(test_y)
print(yhat.shape)
yhat = yhat.reshape((len(yhat), n_out))
inv_yhat = scaler.inverse_transform(yhat)
print(inv_yhat.shape)
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(inv_y, inv_yhat)
print('Test MAE: %.3f' % mae)
# calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)
from sklearn.metrics import r2_score
print('r2 score: %.3f' % r2_score(inv_y, inv_yhat))